## Author name:

<pre> Babak Ansari-Jaberi</pre>
<pre> Calculating the duration of a work item </pre>

# User Story points(weight) prediction

### Data Prepration

Load data from development tracking software 


In [5]:
# pip install jira
from jira import JIRA
import getpass
import pandas as pd
import numpy as np
import os.path
from pandas import ExcelWriter
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.utils.fixes import signature
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
import warnings
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
warnings.filterwarnings('ignore')


file_name = 'persistance/StoryDuration.xlsx'
sheetname = 'User Story Duration'

jiraFileExist = os.path.exists(file_name)

if not jiraFileExist:
    #input( "Username: ")
    userName = 'bob.ansari@intelex.com' 
    password = getpass.getpass(prompt='Username: '+userName+'\r\nPassword: ')


    auth_jira = JIRA(server = 'https://intelex.atlassian.net', auth=( userName, password ))

    # my top 5 issues due by the end of the week, ordered by priority
    oh_crap = auth_jira.search_issues('"Tech Team"="Team 1" AND Sprint is not EMPTY AND "Story Points" is not EMPTY and labels is not EMPTY order by priority desc', maxResults=100,  expand='changelog')
    print("User "+userName+" is logged in")

Username: bob.ansari@intelex.com
Password: ········
User bob.ansari@intelex.com is logged in


In [6]:
# Read data source to create data frame of the labels
# Insert labels into data frame
if not jiraFileExist:
    print("Reading from Jira...")
    pd.options.mode.chained_assignment = None  # default='warn'
    history_df = pd.DataFrame(columns=['Key', 'Points', 'Date', 'From', 'To'])
    
    i = 0
    for issue in oh_crap:
        changelog = issue.changelog
        for history in changelog.histories:
            for item in history.items:
                if item.field == 'status':
                    #print( 'Key:' +issue.key + ' Date:' + history.created + ' From:' + item.fromString + ' To:' + item.toString )
                    history_df.loc[i] = [issue.key, issue.fields.customfield_10049, history.created,  item.fromString, item.toString]
                    i = i + 1
    writer = ExcelWriter(file_name)
    history_df.to_excel(writer,sheetname)
    writer.save()
    print("Data loaded from Jira!")
else:
    print("Data loaded from file:",file_name)
    history_df = pd.read_excel(file_name, sheetname)


Reading from Jira...
Data loaded from Jira!


In [14]:
#df.describe()
history_df['Date'] = pd.to_datetime(history_df['Date'])
history_df = history_df.sort_values(by=['Key', 'Date'])
history_df['Duration'] =  history_df['Date'].shift(-1) - history_df['Date']
history_df.head(10)

,Key,Points,Date,From,To,Duration
559,ILX-10147,5.0,2018-07-27 18:24:35.740,New,Ready for Development,30 days 22:12:35.059000
558,ILX-10147,5.0,2018-08-27 16:37:10.799,Ready for Development,In Development,2 days 21:50:02.526000
557,ILX-10147,5.0,2018-08-30 14:27:13.325,In Development,Needs Code Review,11 days 02:48:58.670000
556,ILX-10147,5.0,2018-09-10 17:16:11.995,Needs Code Review,Ready for Testing,0 days 00:04:10.421000
555,ILX-10147,5.0,2018-09-10 17:20:22.416,Ready for Testing,In Testing,0 days 01:43:17.723000
554,ILX-10147,5.0,2018-09-10 19:03:40.139,In Testing,Ready for Review,9 days 19:27:18.300000
553,ILX-10147,5.0,2018-09-20 14:30:58.439,Ready for Review,Done,27 days 22:05:01.286000
552,ILX-10319,3.0,2018-10-18 12:35:59.725,New,Ready for Development,0 days 00:00:07.721000
551,ILX-10319,3.0,2018-10-18 12:36:07.446,Ready for Development,In Development,4 days 03:20:09.588000
550,ILX-10319,3.0,2018-10-22 15:56:17.034,In Development,Needs Code Review,6 days 22:20:06.191000


In [15]:
# Create DataFrame for States
states_df = history_df[['From','To']].copy()
states_from_vw = history_df[['From']].drop_duplicates()
states_from_vw.columns = ['State']
states_to_vw  = history_df[['To']].drop_duplicates()
states_to_vw.columns = ['State']
states_df = pd.concat([states_from_vw, states_to_vw]).drop_duplicates()

In [16]:
states_df.head(100)

,State
559,New
558,Ready for Development
557,In Development
556,Needs Code Review
555,Ready for Testing
554,In Testing
553,Ready for Review
482,Done
470,Will not implement
79,Accepted


In [20]:
dev_df = history_df.loc[(history_df['From'] == 'In Development') & (history_df['To'] == 'Needs Code Review')]
dev_df['Type'] = 'Dev'
qa_df = history_df.loc[(history_df['From'] == 'In Testing') & (history_df['To'] == 'Ready for Review')]
qa_df['Type'] = 'QA'
po_df = history_df.loc[((history_df['From'] == 'New') & (history_df['To'] == 'Ready for Development')) |
                       ((history_df['From'] == 'Ready for Review') & (history_df['To'] == 'Done'))]
po_df['Type'] = 'PO'
delay_df = history_df.loc[(history_df['From'] == 'Ready for Development') & (history_df['To'] == 'In Development')]
delay_df['Type'] = 'Delay'

progress_df = pd.concat([ dev_df, qa_df, po_df, delay_df ]).sort_values(by=['Key', 'Date'])
duration_df = progress_df[['Key', 'Points', 'Type', 'Duration']].groupby(['Key', 'Points', 'Type']).sum().sort_values(by=['Key'])

#duration_df.head()
progress_df.head()

,Key,Points,Date,From,To,Duration,Type
559,ILX-10147,5.0,2018-07-27 18:24:35.740,New,Ready for Development,30 days 22:12:35.059000,PO
558,ILX-10147,5.0,2018-08-27 16:37:10.799,Ready for Development,In Development,2 days 21:50:02.526000,Process
557,ILX-10147,5.0,2018-08-30 14:27:13.325,In Development,Needs Code Review,11 days 02:48:58.670000,Dev
554,ILX-10147,5.0,2018-09-10 19:03:40.139,In Testing,Ready for Review,9 days 19:27:18.300000,QA
553,ILX-10147,5.0,2018-09-20 14:30:58.439,Ready for Review,Done,27 days 22:05:01.286000,PO


In [31]:
#progress_df.loc[progress_df['Type'] == 'PO'].describe()

In [29]:
progress_df.head()

,Key,Points,Date,From,To,Duration,Type
559,ILX-10147,5.0,2018-07-27 18:24:35.740,New,Ready for Development,30 days 22:12:35.059000,PO
558,ILX-10147,5.0,2018-08-27 16:37:10.799,Ready for Development,In Development,2 days 21:50:02.526000,Process
557,ILX-10147,5.0,2018-08-30 14:27:13.325,In Development,Needs Code Review,11 days 02:48:58.670000,Dev
554,ILX-10147,5.0,2018-09-10 19:03:40.139,In Testing,Ready for Review,9 days 19:27:18.300000,QA
553,ILX-10147,5.0,2018-09-20 14:30:58.439,Ready for Review,Done,27 days 22:05:01.286000,PO


In [32]:
df = progress_df[['Key','Points','Duration','Type']].copy()
df['Duration'] = df['Duration'] / np.timedelta64(1, 'D')
df = df.loc[ df['Duration'] >1 ]
# sns.relplot(x="Points", y="Duration", hue="Type", size="Points",
#             sizes=(40, 400), alpha=.5, palette="muted",
#             height=6, data=df)
#sns.pairplot(df);
#df
df.loc[df['Type'] == 'PO'].describe()

,Points,Duration
count,47.000000,47.000000
mean,4.063830,17.672087
std,2.150877,21.599988
min,1.000000,1.022149
25%,3.000000,2.036719
50%,3.000000,8.134500
75%,5.000000,24.746837
max,8.000000,77.896855


In [52]:
#df.sort_values(by=['Duration', 'Key'])
progress_df.sort_values(by=['Key', 'Duration'])
#history_df.loc[(history_df['From'] == 'In Development') & (history_df['To'] == 'Needs Code Review')]
#qa_df.sort_values(by=['Duration', 'Key'])

,Key,Points,Date,From,To,Duration,Type
535,ILX-10147,5,2018-09-10 19:03:40.139,In Testing,Ready for Review,9 days 19:27:18.300000,QA
538,ILX-10147,5,2018-08-30 14:27:13.325,In Development,Needs Code Review,11 days 02:48:58.670000,Dev
528,ILX-10319,3,2018-10-30 15:11:39.165,In Development,Needs Code Review,2 days 00:39:03.117000,Dev
525,ILX-10319,3,2018-11-09 21:44:31.947,In Testing,Ready for Review,2 days 19:42:12.992000,QA
531,ILX-10319,3,2018-10-22 15:56:17.034,In Development,Needs Code Review,6 days 22:20:06.191000,Dev
518,ILX-10320,3,2018-11-28 20:59:52.202,In Testing,Ready for Review,0 days 00:42:46.856000,QA
521,ILX-10320,3,2018-11-23 16:58:12.567,In Development,Needs Code Review,0 days 01:59:22.816000,Dev
511,ILX-10352,2,2018-08-08 14:04:18.837,In Testing,Ready for Review,0 days 00:48:51.979000,QA
514,ILX-10352,2,2018-07-31 13:58:17.456,In Development,Needs Code Review,7 days 00:54:36.253000,Dev
504,ILX-10356,3,2018-11-15 20:39:33.693,In Testing,Ready for Review,3 days 18:03:59.131000,QA
